In [1]:
from pathlib import Path
import pandas as pd
import json
from os.path import join, exists
import matplotlib.pyplot as plt
import numpy as np


from utils.simulation.downsampling import downsample
from utils.plot import create_gaze_video
from PIL import Image

In [2]:


class ETRA:
    def __init__(self,data_path = 'data\\ETRA2019Challenge'):
        self.data_path = data_path
        self.ptoa = 1/25.47
        dataset = []
        index_path = join(data_path,'index.json')
        if exists(index_path):
            with open(index_path, 'r') as f:
                dataset = json.load(f)
        else:
            dataset = self.__build_dataset()
            with open(index_path,'w') as f:
                json.dump(dataset,f)
        self.index = pd.DataFrame(dataset)
        self.distance_to_screen = 57
        self.img_shape = (630, 921, 3)

    def __build_dataset(self):
        dataset = []
        id_to_path = dict()
        data_path = Path(self.data_path)
        for path in data_path.rglob('*.csv'):
            name = path.name
            if 'Data' in name:
                continue
            id = name.split('.')[0].split('_')
            sample = {
                'subject': id[0],
                'stimulus': id[1],
                'task': id[2],
                'condition': id[3],
                'condition_id': id[4],
                'gaze_path': str(path)
            }
            cond = sample['condition_id']
            if cond == 'grey':
                sample['img_path'] = 'grey'
            elif cond == '':
                sample['img_path'] = None
                sample['condition_id'] = None
            else:
                img_path = id_to_path.get(sample['condition_id'])
                if img_path is None:
                    img_path = str(next(data_path.rglob(f'{cond}.bmp')))
                sample['img_path'] = img_path
            dataset.append(sample)
        return dataset
    

    def __len__(self):
        return len(self.index)
    
    def summary(self):
        print('number of samples', len(self.index))
        print('number of subjects', self.index['subject'].unique().shape[0])
        stimulus_count = [n for n in self.index.groupby('subject')['stimulus'].size()]
        print('stimulus per subject', stimulus_count[0])
        print('task: freeview and fixation')
        print('conditions: Grey, Puzzle, Waldo, Natural')
        print('img per condition: 15')
        print('4 conditions * 2 tasks * 15 images = 120')

    def get_eyetrack(self, idx):
        path_list = self.index['gaze_path'].iloc[idx].to_list()
        return [ETRA.__read_gaze(path) for path in path_list]

    def get_img(self, idx):
        path_list = self.index['img_path'].iloc[idx].to_list()
        img_list = []
        for path in path_list:
            print(path)
            if path == 'grey':
                img_list.append(np.ones(self.img_shape, dtype = np.uint8)*128)
            elif path is None:
                img_list.append(None)
            else:
                img = Image.open(path)
                img_list.append(np.array(img))
        return img_list
            
    def __read_gaze(path):
        data = pd.read_csv(path)
        data =  data[['RXpix','RYpix','Time']].to_numpy().T
        data[2] = data[2] - data[2][0]
        return data
    
        


In [3]:
dataset = ETRA()
dataset.summary()

number of samples 960
number of subjects 8
stimulus per subject 120
task: freeview and fixation
conditions: Grey, Puzzle, Waldo, Natural
img per condition: 15
4 conditions * 2 tasks * 15 images = 120


In [4]:
idx = np.random.randint(0,len(dataset),4)
print(idx)
gaze_list = dataset.get_eyetrack(list(idx))
img_list = dataset.get_img(list(idx))

[849 555 851 675]
data\ETRA2019Challenge\images\NATURAL\nat005.bmp
data\ETRA2019Challenge\images\PUZZLE\puz001.bmp
grey
data\ETRA2019Challenge\images\NATURAL\nat001.bmp


In [5]:
for img in img_list:
    print(img.shape)

(630, 921, 3)
(630, 921, 3)
(630, 921, 3)
(630, 921, 3)


In [6]:
down_gaze_list = downsample(gaze_list, 16.666)

In [7]:
down_gaze_list[0].dtype

dtype('float64')

In [8]:
create_gaze_video(down_gaze_list,img_list,'ETRA')

c:\Users\ulloa\OneDrive\Desktop\Practicas\projectes\Dataset\utils\plot.py:110: RuntimeWarning: invalid value encountered in cast
  coords = gaze_trajectory[:2, :].astype(int) # Use integer coordinates for drawing


✅ Video successfully created at: ETRA_0.mp4
✅ Video successfully created at: ETRA_1.mp4
✅ Video successfully created at: ETRA_2.mp4
✅ Video successfully created at: ETRA_3.mp4


In [9]:
for j, gaze in enumerate(down_gaze_list):

    print('>>>>>> trajectory: ', j, '<<<<<<<<<')
    nan_gaze = np.isnan(gaze)
    nan_agg = np.sum(nan_gaze,axis = 0)
    idx = np.arange(nan_agg.shape[0])
    idx = idx[nan_agg > 0]
    nan_agg = nan_agg[nan_agg > 0]
    print()
    for i in range(idx.shape[0]):
        print('index: ', idx[i], ' val: ', nan_agg[i])
    



>>>>>> trajectory:  0 <<<<<<<<<

index:  356  val:  2
index:  357  val:  2
index:  358  val:  2
index:  359  val:  2
index:  360  val:  2
index:  361  val:  2
index:  362  val:  2
index:  363  val:  2
index:  648  val:  2
index:  649  val:  2
index:  650  val:  2
index:  651  val:  2
index:  652  val:  2
index:  653  val:  2
index:  1721  val:  2
index:  1722  val:  2
index:  1723  val:  2
index:  1724  val:  2
index:  1725  val:  2
index:  1726  val:  2
index:  1727  val:  2
index:  1728  val:  2
index:  1731  val:  2
index:  1732  val:  2
index:  1733  val:  2
index:  1734  val:  2
index:  1735  val:  2
index:  1736  val:  2
index:  1737  val:  2
index:  1738  val:  2
index:  1739  val:  2
index:  1740  val:  2
index:  1741  val:  2
index:  2156  val:  2
index:  2157  val:  2
index:  2158  val:  2
index:  2159  val:  2
index:  2160  val:  2
index:  2161  val:  2
index:  2162  val:  2
index:  2163  val:  2
index:  2164  val:  2
index:  2165  val:  2
>>>>>> trajectory:  1 <<<<<<<<<

in

In [10]:
print(gaze_list[0][0][np.logical_not(np.isnan(gaze_list[0][0]))].min())
print(gaze_list[0][0][np.logical_not(np.isnan(gaze_list[0][0]))].max())
print(gaze_list[0][1][np.logical_not(np.isnan(gaze_list[0][1]))].min())
print(gaze_list[0][1][np.logical_not(np.isnan(gaze_list[0][1]))].max())

360.98
522.34
227.025
618.075


In [6]:
from datetime import datetime

start = datetime(2025,11,1)
end = datetime(2026,1,30)
delta = (start - end)
delta

datetime.timedelta(days=-90)